In [25]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
# Create the agent
memory = MemorySaver()
model = ChatOpenAI(
    api_key="ollama",
    model="qwq",
    base_url="http://localhost:11434/v1",
)

@tool
def find_providers_given_origin_and_destination(origin: str, destination: str) -> str:
    """Check if there is a path between origin and destination addresses and returns the provider names.
    This function makes an API call to find transportation providers that can service a specific route.
    
    The API call includes several fixed parameters:
    - A specific departure and return time on March 20, 2024
    - Eligibility criteria for seniors and those with disabilities
    - Requirement for wheelchair accessibility
    - Need for a companion
    - Acceptance of shared rides
    
    Args:
        origin: starting address for the transportation route
        destination: ending address for the transportation route
        
    Returns:
        list: A list of provider names that can service the specified route with the given requirements
    """
    
    # Make POST request to provider matching API endpoint
    response = requests.post(
        'https://optimat-db.onrender.com/api/v1/providers/match',
        json={
            "departureTime": "2024-03-20T09:30:00-07:00",
            "returnTime": "2024-03-20T14:45:00-07:00",
            "originAddress": origin,
            "destinationAddress": destination,
            "eligibility": ["senior", "disability"],
            "equipment": ["wheelchair"],
            "healthConditions": ["none"],
            "needsCompanion": True,
            "allowsSharing": True
}
    )
    # Parse JSON response
    response_json = response.json()
    # Extract and return just the provider names from the response data
    return [provider["provider_name"] for provider in response_json["data"]]

tools = [find_providers_given_origin_and_destination]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "123"}}
ASSISTANCE_PROMPT = """
You are a helpful assistant developed by OPTIMAT, a team that provides transportation services for people with disabilities and seniors.
You are able to find paratransit providers that can provide services between origin and destination addresses, and other criteria.

Your goal is to ask for the origin and destination addresses, and other criteria, and then find the paratransit providers that can provide services between the origin and destination addresses, and other criteria.
Please do not make up information, only use the information provided by the user.
"""

messages = [SystemMessage(content=ASSISTANCE_PROMPT)]
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! What can you help me with?")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Can i go from 1103 S California Blvd, Walnut Creek, CA 94596 to 1275 Broadway Plaza, Walnut Creek, CA 94596? If so, give me the provider names")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hey Bob! I'm here to assist you with any questions or tasks you might have. Whether it's finding information, solving a problem, or just chatting, feel free to let me know how I can help.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 322, 'total_tokens': 366, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwq', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-965cbb8b-c6d7-447a-8461-814266455be4-0', usage_metadata={'input_tokens': 322, 'output_tokens': 44, 'total_tokens': 366, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6guttemh', 'function': {'arguments': '{"destination":"1275 Broadway Plaza, Walnut Creek, CA 94596","origin":"1103 S California Blvd, Walnut Creek, CA 94596"}', 'name': 'find_p